## Individual Recommender system
Running the cell below you can actually put the individual recommender system in practice. Do not add any actual code about the recommenders in this notebook, do that in 'recommender.ipynb' and then import the necessary methods you need. (ask chatgpt how to do so) 

In [95]:
from surprise import Dataset, Reader, SVD
from collections import defaultdict


In [10]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import keyboard
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor

In [182]:
ratings_df = pd.read_csv('Data/filtered_ratings.csv')
movies = pd.read_csv('Data/movies.csv')

In [103]:
def get_movie_name(movie_id):
    movie_row = movies[movies['movieId'] == movie_id]

    if not movie_row.empty:
        return movie_row['title'].iloc[0]
    else:
        return "Movie not in the list"

In [200]:
def newUser(movie_ratings):
    new_user_id = 1
    
    # Find the next available user ID
    with open('Data/filtered_ratings.csv', mode='r', newline='') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header row
        existing_user_ids = set()
        for row in csv_reader:
            existing_user_ids.add(int(row[0]))
        while new_user_id in existing_user_ids:
            new_user_id += 1

    # Write the new user's ratings to the CSV file, each in a separate row
    with open('Data/filtered_ratings.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        for movie_id, rating in movie_ratings.items():
            writer.writerow([new_user_id, movie_id, rating])
    return new_user_id

In [220]:
import random
def recommendKNN(movie_ratings, new_user_id):
    # Filter users with more than 200 ratings
#     user_ratings = pd.Series(movie_ratings)
#     users_ratings = ratings_df.groupby('userId').count()
#     selected_users = users_ratings[users_ratings['rating'] > 200].index.tolist()
    
    # Select a random user
#     selected_user = random.choice(selected_users)
    selected_user = new_user_id
    
    # Filter ratings for the selected user
    selected_user_ratings = ratings_df[ratings_df['userId'] == selected_user]
    
    # Merge ratings with movie data
    rated_movies_df = movies.merge(selected_user_ratings, on='movieId', how='inner')
    
    # Create TF-IDF vectors for movie genres
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(rated_movies_df['genres'])
    y = rated_movies_df['rating']
    
    # Train KNN Regressor
    neighbors = 0
    if(len(movie_ratings)< 5):
       neighbors = len(movie_ratings)
    nighbors = 5
    neigh = KNeighborsRegressor(n_neighbors=neighbors)
    neigh.fit(X, y)
    
    # Predict ratings for unrated movies
    unrated_movies = movies[~movies['movieId'].isin(rated_movies_df['movieId'])]
    X_unrated = vectorizer.transform(unrated_movies['genres'])
    y_unrated = neigh.predict(X_unrated)
    
    # Create a dictionary with movie IDs and predicted ratings
    predicted_ratings = dict(zip(unrated_movies['movieId'], y_unrated))
    
    # Combine user ratings and predicted ratings
    user_and_predicted_ratings = {**user_ratings, **predicted_ratings}
    
    # Sort the movies based on the combined ratings
    sorted_movies = sorted(user_and_predicted_ratings.items(), key=lambda x: x[1], reverse=True)
    common_movie_count = 0
#     for i, (movie_id, rating) in enumerate(sorted_movies[common_movie_count:10+ common_movie_count], 1):
    for i, (movie_id, rating) in enumerate(sorted_movies, 1):
        if movie_id in movie_ratings:
            common_movie_count += 1
#             print(f"{i}. Movie : {get_movie_name(movie_id)}, Rating: {rating}")
#             print(f"{i}. Movie : {get_movie_name(movie_id)}")
    for i, (movie_id, rating) in enumerate(sorted_movies[common_movie_count:10+ common_movie_count], 1):
#         if movie_id not in movie_ratings:
#             common_movie_count += 1
#             print(f"{i}. Movie : {get_movie_name(movie_id)}, Rating: {rating}")
        print(f"{i}. Movie : {get_movie_name(movie_id)}")
            
    return sorted_movies

In [221]:

movie_ratings = {}

while True:
    movie_name = input("Enter a movie name (or 'done' to finish): ")
    
    if movie_name.lower() == 'done' or keyboard.is_pressed('esc'):
        break
    
    matched_movies = process.extractOne(movie_name, movies['title'], scorer=fuzz.token_set_ratio)

    if matched_movies[1] >= 80:  # Similarity threshold
        movie_id = movies[movies['title'] == matched_movies[0]]['movieId'].values[0]
        correct_movie_name = matched_movies[0]
        while True:
            rating = input(f"Enter a rating for '{correct_movie_name}' (1-5): ")
            try:
                rating = float(rating)
                if 1 <= rating <= 5:
                    break
               
                else:
                    print("Rating must be between 1 and 5.")
            except ValueError:
                print("Invalid rating. Please enter a number between 1 and 5.")
        
        movie_ratings[movie_id] = rating
        print(f"Rating for '{correct_movie_name}' added.")
    else:
        print(f"Movie '{movie_name}' not found in the database.")
    
if movie_ratings:
    while True:
        recommendation_type = input("Enter 'genre' for genre-based recommendation or 'knn' for similar user recommendation: ").lower()
        if recommendation_type == 'genre':

            #----------------------------ADD YOUR WORK HERE-----------------------------
            print('ADD HERE YOUR GENRE-BASED RECOMMENDATIONS BASED ON \'movie_ratings\'')
            print('ADD HERE YOUR EXPLANATION ON THE MOVIE RECOMMENDATIONS')
            #----------------------------ADD YOUR WORK HERE-----------------------------

        elif recommendation_type == 'knn':

            #----------------------------ADD YOUR WORK HERE-----------------------------
            user = newUser(movie_ratings)
            ratings_df = pd.read_csv('Data/filtered_ratings.csv')
            recommendKNN(movie_ratings,user)
#             print('ADD HERE YOUR EXPLANATION ON THE MOVIE RECOMMENDATIONS')
            #----------------------------ADD YOUR WORK HERE-----------------------------

        else:
            print("Invalid choice. Please enter 'genre' or 'user'.")
        break


# print("MovieId-Rating Dictionary:")
print(movie_ratings)


Enter a movie name (or 'done' to finish): toy story
Enter a rating for 'Toy Story (1995)' (1-5): 5
Rating for 'Toy Story (1995)' added.
Enter a movie name (or 'done' to finish): finding nemo
Enter a rating for 'Finding Nemo (2003)' (1-5): 4
Rating for 'Finding Nemo (2003)' added.
Enter a movie name (or 'done' to finish): termiantor
Movie 'termiantor' not found in the database.
Enter a movie name (or 'done' to finish): 1
Enter a rating for 'Naked Gun 33 1/3: The Final Insult (1994)' (1-5): 4
Rating for 'Naked Gun 33 1/3: The Final Insult (1994)' added.
Enter a movie name (or 'done' to finish): knn
Enter a rating for 'Shed No Tears (Känn ingen sorg) (2013)' (1-5): 4
Rating for 'Shed No Tears (Känn ingen sorg) (2013)' added.
Enter a movie name (or 'done' to finish): done
Enter 'genre' for genre-based recommendation or 'knn' for similar user recommendation: knn
1. Movie : Terminator 2: Judgment Day (1991)
2. Movie : Jumanji (1995)
3. Movie : Grumpier Old Men (1995)
4. Movie : Waiting to Ex

### Indivual Recimmender using KNN

In [78]:
users_ratings = ratings_df.groupby(['userId']).count()

selected = users_ratings['rating'] > 200
selected_users = users_ratings.loc[selected]
random_selected = selected_users.sample() # sample() returns a random row from the dataframe. The returned object is a dataframe with one row. If you pass a number as argument you specify to select more than one row.
select_column_df = random_selected.reset_index()['userId'] # reset_index() create a new index, and the user became a column. Then, we can filter using the column name
selected_user = select_column_df.iloc[0] # iloc select by index, since our dataframe only has one row we read it from the index 0
# selected_user = 19
print("Selected user: " + str(selected_user))

Selected user: 9834


In [79]:
selected_user_ratings = ratings_df.loc[ratings_df['userId'] == selected_user]
selected_user_ratings['item'] = selected_user_ratings.index 
selected_user_ratings.to_csv('selected_user_ratings.csv', index=False)
selected_user_ratings = selected_user_ratings.sort_values(by='item', ascending=True)
print("Rated movies: " + str(selected_user_ratings.shape[0]))
# display(selected_user_ratings.head(10))

Rated movies: 780


<ipython-input-79-42af8ea5343e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_user_ratings['item'] = selected_user_ratings.index


In [80]:
selected_movie_ids = set(selected_user_ratings['movieId'])
rated_movies_df = movies.reindex(list(selected_user_ratings['movieId']))
# rated_movies_df = movies_df[movies_df['movieId'].isin(selected_movie_ids)]
rated_movies_df = rated_movies_df[['title', 'genres']]
rated_movies_df['item'] = rated_movies_df.index 
rated_movies_df.to_csv('rated_movies_df.csv', index=False)

In [81]:
diff = set(movies.index) - set(rated_movies_df.index)
unrated_movies_df = movies.loc[diff]
# display(unrated_movies_df.head())
unrated_movies_df = unrated_movies_df[['title', 'genres']]
print("Unrated movies: " + str(unrated_movies_df.shape[0]))
# display(unrated_movies_df.head(10))

Unrated movies: 26705


In [82]:
rated_movies_df = rated_movies_df.join(selected_user_ratings.set_index('movieId')['rating'], on='item')
print("Rated movies: " + str(rated_movies_df.shape[0]))
# display(rated_movies_df.head(10))

Rated movies: 780


In [83]:
rated_movies_df['genres'].fillna('', inplace=True)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(rated_movies_df['genres'])
rated_movies_df['genres'].fillna('', inplace=True)
y = rated_movies_df['rating']
neigh = KNeighborsRegressor(n_neighbors=5)
neigh.fit(X, y)
X_unrated = vectorizer.transform(unrated_movies_df['genres'])
y_unrated = neigh.predict(X_unrated)
unrated_movies_df['predicted_ratings_KNN'] = y_unrated


In [84]:
user_ratings = movie_ratings  # Use the provided 'movie_ratings' dictionary

# Train a KNN model to predict ratings
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(rated_movies_df['genres'])
y = rated_movies_df['rating']

neigh = KNeighborsRegressor(n_neighbors=5)
neigh.fit(X, y)

#Predict ratings for unrated movies
X_unrated = vectorizer.transform(unrated_movies_df['genres'])
y_unrated = neigh.predict(X_unrated)

#Dictionary with movie IDs and predicted ratings
predicted_ratings = {}
for idx, movie_id in enumerate(unrated_movies_df.index):
    predicted_ratings[movie_id] = y_unrated[idx]

user_and_predicted_ratings = {**user_ratings, **predicted_ratings}

# Sort the movies based on the combined ratings
sorted_movies = sorted(user_and_predicted_ratings.items(), key=lambda x: x[1], reverse=True)

# Print the top 10 recommended movies
print("Top 10 Recommended Movies:")
for i, (movie_id, rating) in enumerate(sorted_movies[:10], 1):
    print(f"{i}. Movie ID: {movie_id}, Rating: {rating}")
for i, (movie_id, rating) in enumerate(sorted_movies[:10], 1):
    print(f"{i}. Movie : {get_movie_name(movie_id)}, Rating: {rating}")

Top 10 Recommended Movies:
1. Movie ID: 6377, Rating: 5.0
2. Movie ID: 12460, Rating: 4.2
3. Movie ID: 14325, Rating: 4.2
4. Movie ID: 3842, Rating: 4.1
5. Movie ID: 8282, Rating: 4.1
6. Movie ID: 11187, Rating: 4.1
7. Movie ID: 13522, Rating: 4.1
8. Movie ID: 15031, Rating: 4.1
9. Movie ID: 15269, Rating: 4.1
10. Movie ID: 15319, Rating: 4.1
1. Movie : Finding Nemo (2003), Rating: 5.0
2. Movie : Movie not in the list, Rating: 4.2
3. Movie : Movie not in the list, Rating: 4.2
4. Movie : Movie not in the list, Rating: 4.1
5. Movie : Movie not in the list, Rating: 4.1
6. Movie : Movie not in the list, Rating: 4.1
7. Movie : Movie not in the list, Rating: 4.1
8. Movie : Movie not in the list, Rating: 4.1
9. Movie : Movie not in the list, Rating: 4.1
10. Movie : Movie not in the list, Rating: 4.1


## Group Recommender system
In this case I decided not to ask the users for the ratings because it is counterproductive. In this case, the group ratings will be already defined in the code


In [6]:
#Some random group users movie ratings
group_movie_ratings = [
    {1: 5, 2: 3, 13: 4, 3: 4.5},
    {190: 2, 9372: 4, 837: 1.5},
    {89: 3.5, 7521: 3, 90: 3.5}
]

In [7]:
#----------------------------ADD YOUR WORK HERE-----------------------------
print("ADD HERE YOUR MOVIE RECOMMENDATIONS BASED ON \'group_movie_ratings\'")
print('ADD HERE YOUR EXPLANATION ON THE MOVIE RECOMMENDATIONS')
#----------------------------ADD YOUR WORK HERE-----------------------------

ADD HERE YOUR MOVIE RECOMMENDATIONS BASED ON 'group_movie_ratings'
ADD HERE YOUR EXPLANATION ON THE MOVIE RECOMMENDATIONS
